In [10]:
"""Baseline model. 1 layer LSTM with a classification layer trained for 100 epochs"""

'Baseline model. 1 layer LSTM with a classification layer trained for 100 epochs'

In [11]:
import sys
import os
import math

import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv("../../data/processed/signalEUR_USD_Labelled_v1_processed.csv", index_col=0)

In [13]:
df["Date"] = pd.to_datetime(df["Date"])
# Labels 3 days ahead
df["label_3days"] = df["label"].shift(-3)
# Labels 5 days ahead
df["label_5days"] = df["label"].shift(-5)

In [14]:
df = df.dropna(axis = 0)
df = df.reset_index(drop=True)
df.head()

,Date,Price,Open,High,Low,Change %,diff_1,label,sma,ema,...,macd,macd_s,macd_h,roc,rsi,Bollinger_up,Bollinger_down,cci,label_3days,label_5days
0,2013-02-15,1.3363,1.3363,1.3394,1.3306,0.01%,0.0002,0.0,1.344735,1.336730,...,0.002701,0.005461,-0.002760,-0.000150,39.939646,1.364281,1.325189,-62.849696,1.0,0.0
1,2013-02-18,1.3352,1.3357,1.3377,1.3321,-0.08%,-0.0011,0.0,1.344925,1.335553,...,0.001906,0.004750,-0.002844,-0.004102,38.588551,1.363994,1.325856,-68.561252,1.0,1.0
2,2013-02-19,1.3388,1.3352,1.3397,1.3329,0.27%,0.0036,2.0,1.345260,1.338051,...,0.001549,0.004109,-0.002561,-0.004906,45.544851,1.363601,1.326919,-57.039491,0.0,0.0
3,2013-02-20,1.3281,1.3387,1.3434,1.3270,-0.80%,-0.0107,1.0,1.345075,1.330396,...,0.000398,0.003367,-0.002969,-0.012785,32.802173,1.364052,1.326098,-90.131403,1.0,2.0
4,2013-02-21,1.3189,1.3283,1.3291,1.3161,-0.69%,-0.0092,1.0,1.344140,1.321553,...,-0.001242,0.002445,-0.003688,-0.012873,25.675429,1.366251,1.322029,-149.263447,0.0,1.0


In [15]:
time = ["Date"]

columns = ["Price",  "sma", "ema", "cma", "macd",	"macd_s", 
           "macd_h", "roc", "rsi",	"Bollinger_up", "Bollinger_down", "cci"]

labels = ["label", "label_3days", "label_5days"]

features = df[time + columns + labels]

In [16]:
features.head()

,Date,Price,sma,ema,cma,macd,macd_s,macd_h,roc,rsi,Bollinger_up,Bollinger_down,cci,label,label_3days,label_5days
0,2013-02-15,1.3363,1.344735,1.336730,1.335315,0.002701,0.005461,-0.002760,-0.000150,39.939646,1.364281,1.325189,-62.849696,0.0,1.0,0.0
1,2013-02-18,1.3352,1.344925,1.335553,1.335311,0.001906,0.004750,-0.002844,-0.004102,38.588551,1.363994,1.325856,-68.561252,0.0,1.0,1.0
2,2013-02-19,1.3388,1.345260,1.338051,1.335408,0.001549,0.004109,-0.002561,-0.004906,45.544851,1.363601,1.326919,-57.039491,2.0,0.0,0.0
3,2013-02-20,1.3281,1.345075,1.330396,1.335211,0.000398,0.003367,-0.002969,-0.012785,32.802173,1.364052,1.326098,-90.131403,1.0,1.0,2.0
4,2013-02-21,1.3189,1.344140,1.321553,1.334782,-0.001242,0.002445,-0.003688,-0.012873,25.675429,1.366251,1.322029,-149.263447,1.0,0.0,1.0


In [22]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

X = features.drop(labels= time + labels, axis=1)
Y = features[["label"]]

scaler = StandardScaler()
X = scaler.fit_transform(X)

onehot_encoder = OneHotEncoder()
Y = to_categorical(Y)

In [23]:
print(Y.shape)

(2311, 3)


In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=420)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(1848, 12) (463, 12)
(1848, 3) (463, 3)


In [25]:
from keras.models import Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.losses import CategoricalCrossentropy

h_dim = 100
num_classes = 3

model = Sequential()
model.add(LSTM(units=h_dim, input_shape=(X_train.shape[1], 1)))
model.add(Dense(num_classes))
model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               40800     
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 41,103
Trainable params: 41,103
Non-trainable params: 0
_________________________________________________________________


In [26]:
epochs = 100

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64)

Epoch 1/100
29/29 [==============================] - 2s 19ms/step - loss: 1.0763 - accuracy: 0.4156 - val_loss: 1.0764 - val_accuracy: 0.4039
Epoch 2/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0496 - accuracy: 0.4551 - val_loss: 1.0660 - val_accuracy: 0.4147
Epoch 3/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0311 - accuracy: 0.4594 - val_loss: 1.0567 - val_accuracy: 0.4341
Epoch 4/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0129 - accuracy: 0.4643 - val_loss: 1.0418 - val_accuracy: 0.4838
Epoch 5/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0075 - accuracy: 0.4665 - val_loss: 1.0477 - val_accuracy: 0.4384
Epoch 6/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0051 - accuracy: 0.4729 - val_loss: 1.0446 - val_accuracy: 0.4687
Epoch 7/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0072 - accuracy: 0.4708 - val_loss: 1.0524 - val_accuracy: 0.4557
Epoch 8/100


Epoch 59/100
29/29 [==============================] - 0s 8ms/step - loss: 0.8052 - accuracy: 0.6104 - val_loss: 0.8242 - val_accuracy: 0.6004
Epoch 60/100
29/29 [==============================] - 0s 8ms/step - loss: 0.8075 - accuracy: 0.6190 - val_loss: 0.8437 - val_accuracy: 0.5896
Epoch 61/100
29/29 [==============================] - 0s 8ms/step - loss: 0.7981 - accuracy: 0.6250 - val_loss: 0.8333 - val_accuracy: 0.5940
Epoch 62/100
29/29 [==============================] - 0s 8ms/step - loss: 0.7949 - accuracy: 0.6212 - val_loss: 0.8286 - val_accuracy: 0.6048
Epoch 63/100
29/29 [==============================] - 0s 8ms/step - loss: 0.7889 - accuracy: 0.6212 - val_loss: 0.8233 - val_accuracy: 0.6091
Epoch 64/100
29/29 [==============================] - 0s 8ms/step - loss: 0.8032 - accuracy: 0.6109 - val_loss: 0.8993 - val_accuracy: 0.5443
Epoch 65/100
29/29 [==============================] - 0s 8ms/step - loss: 0.7964 - accuracy: 0.6190 - val_loss: 0.8139 - val_accuracy: 0.6048
Epoch 

In [7]:
# Without price as a factor

In [30]:
X = features.drop(labels= time + labels + ["Price"], axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[ 1.84392654,  1.76562961,  1.70005358, ...,  1.8683638 ,
         1.80152324, -0.65041763],
       [ 1.84605032,  1.75246368,  1.69999683, ...,  1.8651378 ,
         1.80889088, -0.71655935],
       [ 1.8497949 ,  1.78040628,  1.70167462, ...,  1.86074147,
         1.82061006, -0.5831335 ],
       ...,
       [-0.56483883, -0.53162927, -0.96953338, ..., -0.69084061,
        -0.43521197,  0.62254833],
       [-0.55802035, -0.51948747, -0.96989049, ..., -0.69725765,
        -0.41543563,  0.97031967],
       [-0.55802035, -0.5261519 , -0.97025541, ..., -0.69725765,
        -0.41543563,  0.92950638]])

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=420)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(1848, 11) (463, 11)
(1848, 3) (463, 3)


In [32]:
model = Sequential()
model.add(LSTM(units=h_dim, input_shape=(X_train.shape[1], 1)))
model.add(Dense(num_classes))
model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100)               40800     
                                                                 
 dense_2 (Dense)             (None, 3)                 303       
                                                                 
Total params: 41,103
Trainable params: 41,103
Non-trainable params: 0
_________________________________________________________________


In [33]:
epochs = 100

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64)

Epoch 1/100
29/29 [==============================] - 2s 18ms/step - loss: 1.0756 - accuracy: 0.3896 - val_loss: 1.0765 - val_accuracy: 0.3996
Epoch 2/100
29/29 [==============================] - 0s 7ms/step - loss: 1.0473 - accuracy: 0.4443 - val_loss: 1.0573 - val_accuracy: 0.4320
Epoch 3/100
29/29 [==============================] - 0s 7ms/step - loss: 1.0238 - accuracy: 0.4740 - val_loss: 1.0522 - val_accuracy: 0.4536
Epoch 4/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0126 - accuracy: 0.4665 - val_loss: 1.0416 - val_accuracy: 0.4644
Epoch 5/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0064 - accuracy: 0.4773 - val_loss: 1.0516 - val_accuracy: 0.4644
Epoch 6/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0044 - accuracy: 0.4751 - val_loss: 1.0406 - val_accuracy: 0.4622
Epoch 7/100
29/29 [==============================] - 0s 8ms/step - loss: 0.9992 - accuracy: 0.4854 - val_loss: 1.0460 - val_accuracy: 0.4428
Epoch 8/100


Epoch 59/100
29/29 [==============================] - 0s 8ms/step - loss: 0.7917 - accuracy: 0.6239 - val_loss: 0.8371 - val_accuracy: 0.5810
Epoch 60/100
29/29 [==============================] - 0s 8ms/step - loss: 0.7968 - accuracy: 0.6239 - val_loss: 0.8238 - val_accuracy: 0.6004
Epoch 61/100
29/29 [==============================] - 0s 9ms/step - loss: 0.7912 - accuracy: 0.6098 - val_loss: 0.8542 - val_accuracy: 0.5745
Epoch 62/100
29/29 [==============================] - 0s 8ms/step - loss: 0.7969 - accuracy: 0.6120 - val_loss: 0.8069 - val_accuracy: 0.6177
Epoch 63/100
29/29 [==============================] - 0s 8ms/step - loss: 0.7906 - accuracy: 0.6299 - val_loss: 0.8810 - val_accuracy: 0.5486
Epoch 64/100
29/29 [==============================] - 0s 8ms/step - loss: 0.7905 - accuracy: 0.6163 - val_loss: 0.8123 - val_accuracy: 0.6177
Epoch 65/100
29/29 [==============================] - 0s 10ms/step - loss: 0.7792 - accuracy: 0.6288 - val_loss: 0.8257 - val_accuracy: 0.5961
Epoch